In [3]:
from math import log

 $l(x_{i}):香农熵,信息的期望值$ 
$$l\left(x_{i}\right)=-\log _{2} p\left(x_{i}\right)$$
$p(x_{i})$:选择该分类的概率 
$$H=-\sum_{i=1}^{n} p\left(x_{i}\right) \log _{2} p\left(x_{i}\right) $$
$H$：所有可能值包含的信息期望值

In [4]:
def calcShanonEnt(dataSet):
    numEntires = len(dataSet)  # 计算数据集实例总数
    labelCounts = {} # 创建字典，
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys:
            labelCounts[currentLabel] = 0 
            labelCounts[currentLabel] += 1 #属于该标签的样本统计数量++
        shannonEnt = 0.0
        for key in labelCounts:
            prob = float(labelCounts[key]/numEntires) #计算概率
            shannonEnt -= prob*log(prob,2) #求对数，概率*信息期望值（熵） 累加 得到信息期望值H
        return shannonEnt

创建数据集

In [5]:
def createDataSet():
    dataSet =[ [1,1, 'yes'],
               [1,1,'yes'],
               [1,0,'no'],
               [0, 1, 'no'],
               [0, 1, 'yes']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [6]:
myDat, labels = createDataSet()
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'yes']]